In [2]:
%load_ext autoreload

%autoreload 2

In [3]:
import os
from glob import glob
import subprocess
import shutil
import traceback
import random
import pickle
from os.path import join

import joblib
from concurrent.futures import ProcessPoolExecutor

from backports import tempfile

import matplotlib
matplotlib.rcParams['savefig.dpi'] = 144
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

import psrchive

import pipe

In [4]:
import logging
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
logging.debug("test")

DEBUG:root:test


In [5]:
!pac -h

A program for calibrating Pulsar::Archives
Usage: pac [options] filenames
  -q             Quiet mode
  -v             Verbose mode
  -V             Very verbose mode
  -i             Show revision information

Database options: 
  -d database    Read calibration database summary 
  -p path        Search for CAL files in the specified path 
  -u ext         Add to file extensions recognized in search 
                 (defaults: .cf .pcal .fcal .pfit) 
  -w             Write a new database summary file 
  -W             Create database from calibrators listed in metafiles 
  -k filename    Output database to filename 
  -l             Cache last calibrator

Calibrator options: 
  -A filename    Use the calibrator in filename, as output by pcm/pacv 
  -C model       Use the specified model of the calibrator solution 
  -P             Calibrate polarisation only (not flux)
  -R             Calibrate the receiver (feed) only 
  -r filename    Use the specified receiver parameters file 
  

In [6]:
cals = []
for d in ["data/raw/GBT", "data/raw/AO", "data/raw/AO/from_UBC", "data/raw/GBT/new/"]:
    cals.extend(glob(join(d,"?uppi_*_cal_*.*")))
for d in ["data/cal/fluxcal/AO", "data/cal/fluxcal/AO/new", "data/cal/fluxcal/GBT"]:
    cals.extend(glob(join(d,"*.fcal")))
cals.extend(glob(join("data","cal","*.cf")))
cals.sort()

with open("data/cal/cal-list.ls", "wt") as f:
    for c in cals:
        f.write(c)
        f.write("\n")

In [7]:
P = subprocess.Popen(["pac",
                 "-w",
                 "-k", "data/cal/cal-new.db",
                 "-W", "data/cal/cal-list.ls"], 
                 stderr=subprocess.PIPE, 
                 stdout=subprocess.PIPE)
out, err = P.communicate()
print out
print err
if P.returncode:
    raise ValueError("Return code %d" % P.returncode)

pac: Generating new calibrator database
pac: Loading calibrator filenames from metafile=data/cal/cal-list.ls
pac: Writing database summary file to data/cal/cal-new.db

Pulsar::Database::add keeping newest of duplicate entries:
	data/cal/fluxcal/AO/new/puppi_57389_J1445+099cal_5481_cal.fcal and
	data/cal/fluxcal/AO/puppi_57389_J1445+099cal_5481_cal.fcal
Pulsar::Database::add keeping newest of duplicate entries:
	data/cal/fluxcal/AO/new/puppi_57389_J1445+099cal_5485_cal.fcal and
	data/cal/fluxcal/AO/puppi_57389_J1445+099cal_5485_cal.fcal
Pulsar::Database::add keeping newest of duplicate entries:
	data/cal/fluxcal/AO/new/puppi_57417_J1445+099cal_5874_cal.fcal and
	data/cal/fluxcal/AO/puppi_57417_J1445+099cal_5874_cal.fcal
Pulsar::Database::add keeping newest of duplicate entries:
	data/cal/fluxcal/AO/new/puppi_57417_J1445+099cal_5878_cal.fcal and
	data/cal/fluxcal/AO/puppi_57417_J1445+099cal_5878_cal.fcal
Pulsar::Database::add keeping newest of duplicate entries:
	data/cal/fluxcal/AO/new/